<a href="https://colab.research.google.com/github/LuisCarlosJaramillo/Vision_Artificial_Deteccion_Clasificacion_Yolov5/blob/main/Visi%C3%B3n_Artificial_Detecci%C3%B3n_Clasificaci%C3%B3n_Yolov5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opencv-python-headless
!pip install yolov5

In [2]:
import torch
import cv2
from google.colab.patches import cv2_imshow

In [ ]:
# Descargar el modelo YOLOv5 pre-entrenado
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

# Asignar etiquetas en español
labels_es = {
    'person': 'persona',
    'cat': 'gato',
    'dog': 'perro',
    'car': 'auto',
    # Añadir más etiquetas si es necesario
}

In [4]:
# Cargar el video
cap = cv2.VideoCapture('/content/drive/MyDrive/Entrenamientos_Vision_Artificial/video_entrenamiento_vision_artificial.mp4')

# Verificar si el video se cargó correctamente
if not cap.isOpened():
    print("Error: No se pudo abrir el video.")
    exit()

# Configurar el codec y crear un objeto VideoWriter para guardar el video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output.mp4', fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))

# Diccionario para contar las detecciones
detections_count = {
    'persona': 0,
    'gato': 0,
    'perro': 0,
    'auto': 0,
    'otros': 0
}

# Loop principal para procesar el video
while True:
    ret, frame = cap.read()

    if not ret:
        break  # Salir si no hay más frames

    # Convertir el frame a RGB para YOLO
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Detección usando YOLOv5
    results = model(frame_rgb)

    # Reiniciar el conteo en cada frame
    detections_count = {key: 0 for key in detections_count}

    # Procesar los resultados
    for result in results.xyxy[0]:
        # Extraer la información de cada objeto detectado
        x1, y1, x2, y2, conf, cls = result

        # Convertir a entero los valores de coordenadas
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

        # Obtener la etiqueta en inglés y en español
        label_eng = model.names[int(cls)]
        label_es = labels_es.get(label_eng, 'otros')  # Usar 'otros' si no está en el diccionario

        # Incrementar el conteo de la etiqueta correspondiente
        if label_es in detections_count:
            detections_count[label_es] += 1
        else:
            detections_count['otros'] += 1

        # Asignar un color diferente para cada clase
        if label_es == 'persona':
            color = (0, 255, 0)  # Verde para personas
        elif label_es in ['gato', 'perro']:
            color = (255, 0, 0)  # Azul para animales
        else:
            color = (0, 0, 255)  # Rojo para otros objetos

        # Dibujar el rectángulo
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)

        # Poner la etiqueta
        cv2.putText(frame, label_es, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

    # Crear un tablero de conteo
    y0, dy = 50, 30
    for i, (key, value) in enumerate(detections_count.items()):
        text = f'{key}: {value}'
        cv2.putText(frame, text, (10, y0 + i * dy), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    # Escribir el frame procesado en el archivo de video
    out.write(frame)

# Liberar el video y cerrar los recursos
cap.release()
out.release()
cv2.destroyAllWindows()

# Descargar el video procesado
from google.colab import files
files.download('output.mp4')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>